In [1]:
import pandas as pd
import numpy as np
import sqlite3
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [2]:
#purpose of this function to match the jobtitles to esco job titles with Levensthein distance. The match here is with the same language
def text_match_same_language(row):
    if row['language'] == 'HU':
        matched_esco = process.extractOne(row['jobtitle'], esco_need_job_titles_hungarian['preferred_occupation_label'].tolist(), scorer=fuzz.token_set_ratio, score_cutoff=85)
        return matched_esco
    elif row['language'] == 'DE':
        matched_esco = process.extractOne(row['jobtitle'], esco_need_job_titles_german['preferred_occupation_label'].tolist(), scorer=fuzz.token_set_ratio, score_cutoff=85)
        return matched_esco
    elif row['language'] == 'ES':
        matched_esco = process.extractOne(row['jobtitle'], esco_need_job_titles_spanish['preferred_occupation_label'].tolist(), scorer=fuzz.token_set_ratio, score_cutoff=85)
        return matched_esco
    elif row['language'] == 'SV':
        matched_esco = process.extractOne(row['jobtitle'], esco_need_job_titles_sweden['preferred_occupation_label'].tolist(), scorer=fuzz.token_set_ratio, score_cutoff=85)
        return matched_esco    
    elif row['language'] == 'EN':
        return row['english_match']

In [3]:
#Function for cosine distance return the language specific results only

def cosine_similarity(language, esco_corpus):
    
    if language == 'EN':
    
        vectorizer = TfidfVectorizer(ngram_range=(1,4), analyzer='char', use_idf=False)
        vectorizer.fit(esco_corpus['preferred_occupation_label'].tolist())

        positions_transformed = vectorizer.transform(list(multi_job['jobtitle']))
        esco_tranformed = vectorizer.transform(esco_corpus['preferred_occupation_label'].tolist())


        trs = 0.7
        sim_mat = positions_transformed * esco_tranformed.T
        sim_mat.data *= sim_mat.data >= trs
        sim_mat.eliminate_zeros()
        sim_mat = sim_mat.tocoo(copy=False)

        position_sim = pd.DataFrame({
        'pos_ix_row': sim_mat.row,
        'pos_ix_col': sim_mat.col,
        'ngram_sim': sim_mat.data})

        position_sim = pd.merge(
        position_sim, multi_job,
        how='left',
        left_on = 'pos_ix_row',
        right_index = True)

        position_sim = pd.merge(
        position_sim, esco_corpus,
        how='left',
        left_on = 'pos_ix_col',
        right_index = True
        )

        cosine_sim_result = 'cosine_sim_result' + '_' + language
        ngram_sim = 'ngram_sim' + '_' + language
        similarity_result = position_sim.loc[position_sim.groupby('uniq_id')['ngram_sim'].idxmax()]
        similarity_result.rename(columns={'preferred_occupation_label' : cosine_sim_result,
                                        'ngram_sim': ngram_sim}, inplace=True)

        multi_job_joined = pd.merge(multi_job, similarity_result[['uniq_id', cosine_sim_result, ngram_sim]], how='left', on='uniq_id')
        return multi_job_joined
    else:
        
        language_df = multi_job[multi_job['language']==language].copy()
        language_df.reset_index(inplace=True)
        del(language_df['index'])
        
        vectorizer = TfidfVectorizer(ngram_range=(1,4), analyzer='char', use_idf=False)
        vectorizer.fit(esco_corpus['preferred_occupation_label'].tolist())

        positions_transformed = vectorizer.transform(list(language_df['jobtitle']))
        esco_tranformed = vectorizer.transform(esco_corpus['preferred_occupation_label'].tolist())


        trs = 0.7
        sim_mat = positions_transformed * esco_tranformed.T
        sim_mat.data *= sim_mat.data >= trs
        sim_mat.eliminate_zeros()
        sim_mat = sim_mat.tocoo(copy=False)

        position_sim = pd.DataFrame({
        'pos_ix_row': sim_mat.row,
        'pos_ix_col': sim_mat.col,
        'ngram_sim': sim_mat.data})

        position_sim = pd.merge(
        position_sim, language_df,
        how='left',
        left_on = 'pos_ix_row',
        right_index = True)

        position_sim = pd.merge(
        position_sim, esco_corpus,
        how='left',
        left_on = 'pos_ix_col',
        right_index = True
        )

        cosine_sim_result = 'cosine_sim_result' + '_' + language
        ngram_sim = 'ngram_sim' + '_' + language
        similarity_result = position_sim.loc[position_sim.groupby('uniq_id')['ngram_sim'].idxmax()]
        similarity_result.rename(columns={'preferred_occupation_label' : cosine_sim_result,
                                        'ngram_sim': ngram_sim}, inplace=True)

        multi_job_joined = pd.merge(language_df, similarity_result[['uniq_id', cosine_sim_result, ngram_sim]], how='left', on='uniq_id')
        return multi_job_joined

In [4]:
#Function for assigning the best possible matches (same language or english) with best scores
def prediction_sorter(row):
    if row['language'] == 'EN':
        if row['english_match'] != 'mv-9':
            return row['english_match']
        else:
            return row['cosine_sim_result_EN']
    elif row['language'] == 'HU':
        if row['same_language_match'] != 'mv-9':
            return row['same_language_match']
        elif row['english_match'] != 'mv-9':
            return row['english_match']
        elif row['cosine_sim_result_HU'] != 'mv-9':
            return row['cosine_sim_result_HU']
        else:
            return row['cosine_sim_result_EN']
    elif row['language'] == 'DE':
        if row['same_language_match'] != 'mv-9':
            return row['same_language_match']
        elif row['english_match'] != 'mv-9':
            return row['english_match']
        elif row['cosine_sim_result_DE'] != 'mv-9':
            return row['cosine_sim_result_DE']
        else:
            return row['cosine_sim_result_EN']
    elif row['language'] == 'ES':
        if row['same_language_match'] != 'mv-9':
            return row['same_language_match']
        elif row['english_match'] != 'mv-9':
            return row['english_match']
        elif row['cosine_sim_result_ES'] != 'mv-9':
            return row['cosine_sim_result_ES']
        else:
            return row['cosine_sim_result_EN']
    elif row['language'] == 'SV':
        if row['same_language_match'] != 'mv-9':
            return row['same_language_match']
        elif row['english_match'] != 'mv-9':
            return row['english_match']
        elif row['cosine_sim_result_SV'] != 'mv-9':
            return row['cosine_sim_result_SV']
        else:
            return row['cosine_sim_result_EN']
    

In [5]:
# if result comes from levenshtein metric then clean it
def prediction_cleaner(row):
    if type(row['final_match']) == tuple:
        return row['final_match'][0]
    else:
        return row['final_match']

In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [7]:
#read in data which was scraped from various sites and the supporting files. Formatting and content differs in scraped data
isco_checked = pd.read_excel('isco_labels_checked.xlsx')
esco_joined_multi_lang = pd.read_csv('esco_joined_data_multi_lang_2018_04_28.csv', sep='^', encoding='utf-8')
english_job = pd.read_csv('dice_com-job_us_sample.csv')
hungarian_job = pd.read_excel('profession_hungarian_positions.xlsx')
german_job = pd.read_csv('german_data_complete_2018-05-02.csv', sep='^', encoding='utf-8', header=None)
spanish_job = pd.read_csv('spanish_data_2018-05-01.csv', sep='^', encoding='utf-8')
swedish_job = pd.read_csv('swedish_data_2018-05-02.csv', sep='^', encoding='utf-8')

C:\Users\cameg\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (6,12,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
#filter esco for IT only
esco_need = esco_joined_multi_lang[esco_joined_multi_lang['iscoGroup'].isin(isco_checked['isco_group'])].copy()
esco_need.reset_index(inplace=True)

In [9]:
#prepare swedish data in unified format, for later merging
swedish_job.columns = ['index', 'jobtitle', 'company', 'joblocation_address', 'Job_Detail_Url']
del(swedish_job['index'])
del(swedish_job['Job_Detail_Url'])
swedish_job['row'] = swedish_job.index
swedish_job['uniq_id'] = swedish_job['jobtitle'] + ':' + swedish_job['company'] + ':' + swedish_job['joblocation_address'] + ':' + swedish_job['row'].astype(str)
del(swedish_job['row'])

In [10]:
#prepare spanish data in unified format, for later merging
spanish_job.columns = ['index', 'jobtitle', 'company', 'joblocation_address', 'Job_Detail_Url']
del(spanish_job['index'])
del(spanish_job['Job_Detail_Url'])
spanish_job['row'] = spanish_job.index
spanish_job['uniq_id'] = spanish_job['jobtitle'] + ':' + spanish_job['company'] + ':' + spanish_job['joblocation_address'] + ':' + spanish_job['row'].astype(str)
del(spanish_job['row'])

In [11]:
#prepare german data in unified format, for later merging
german_job = german_job.iloc[1:]
german_job.columns = ['index', 'jobtitle', 'company', 'joblocation_address', 'Job_Detail_Url']
del(german_job['index'])
del(german_job['Job_Detail_Url'])
german_job['row'] = german_job.index
german_job['uniq_id'] = german_job['jobtitle'] + ':' + german_job['company'] + ':' + german_job['joblocation_address'] + ':' + german_job['row'].astype(str)
del(german_job['row'])

In [12]:
#prepare hungarian data in unified format, for later merging
hungarian_job['joblocation_address'] = 'Budapest'
hungarian_job['row'] = hungarian_job.index
hungarian_job.rename(columns={'Position' : 'jobtitle',
                             'Company' : 'company'}, inplace=True)
hungarian_job['uniq_id'] = hungarian_job['jobtitle'] + ':' + hungarian_job['company'] + ':' + hungarian_job['joblocation_address'] + ':' + hungarian_job['row'].astype(str)
del(hungarian_job['row'])


In [13]:
#assign language filter for scraped job board data
english_job['language'] = 'EN'
hungarian_job['language'] = 'HU'
german_job['language'] = 'DE'
spanish_job['language'] = 'ES'
swedish_job['language'] = 'SV'

In [14]:
#creating multi language job dataset
english_job = english_job[['jobtitle', 'company', 'joblocation_address' , 'uniq_id', 'language']].copy()
multi_job = pd.concat([english_job, hungarian_job, german_job, spanish_job, swedish_job])

In [15]:
#preparing esco data for later matching
esco_need_job_titles = esco_need[['preferred_occupation_label', 'language']].copy()
esco_need_job_titles['preferred_occupation_label'] = esco_need_job_titles['preferred_occupation_label'].apply(lambda text: text.lower())
languages = list(multi_job['language'].drop_duplicates())

In [16]:
#Creating language speficic job titles from esco. This will be the base for the job title matching later on
esco_need_job_titles_hungarian = pd.DataFrame(esco_need_job_titles['preferred_occupation_label'][esco_need_job_titles['language']=='HU'].drop_duplicates())
esco_need_job_titles_english = pd.DataFrame(esco_need_job_titles['preferred_occupation_label'][esco_need_job_titles['language']=='EN'].drop_duplicates())
esco_need_job_titles_german = pd.DataFrame(esco_need_job_titles['preferred_occupation_label'][esco_need_job_titles['language']=='DE'].drop_duplicates())
esco_need_job_titles_spanish = pd.DataFrame(esco_need_job_titles['preferred_occupation_label'][esco_need_job_titles['language']=='ES'].drop_duplicates())
esco_need_job_titles_sweden = pd.DataFrame(esco_need_job_titles['preferred_occupation_label'][esco_need_job_titles['language']=='SV'].drop_duplicates())

In [17]:
#list_of_datasets = [esco_need_job_titles_hungarian, esco_need_job_titles_english, esco_need_job_titles_german , esco_need_job_titles_spanish, esco_need_job_titles_sweden]

In [18]:
#Resetting indexes - Necessary because of join later on in cosine distance

esco_need_job_titles_hungarian.reset_index(inplace=True)
del(esco_need_job_titles_hungarian['index'])

esco_need_job_titles_english.reset_index(inplace=True)
del(esco_need_job_titles_english['index'])

esco_need_job_titles_german.reset_index(inplace=True)
del(esco_need_job_titles_german['index'])

esco_need_job_titles_spanish.reset_index(inplace=True)
del(esco_need_job_titles_spanish['index'])

esco_need_job_titles_sweden.reset_index(inplace=True)
del(esco_need_job_titles_sweden['index'])
   

In [19]:
#esco_need_job_titles_hungarian = esco_need_job_titles_hungarian['preferred_occupation_label'].tolist()
#esco_need_job_titles_english = esco_need_job_titles_english['preferred_occupation_label'].tolist()
#esco_need_job_titles_german = esco_need_job_titles_german['preferred_occupation_label'].tolist()
#esco_need_job_titles_spanish = esco_need_job_titles_spanish['preferred_occupation_label'].tolist()
#esco_need_job_titles_sweden = esco_need_job_titles_sweden['preferred_occupation_label'].tolist()

In [20]:
#lowercasing job titles in scraped data
multi_job['jobtitle'] = multi_job['jobtitle'].apply(lambda text: text.lower())

In [21]:
#formatting of merged job data
multi_job = multi_job.astype(str)
multi_job.reset_index(inplace=True)
del(multi_job['index'])

In [22]:
#Matching for english esco job titles for all records
multi_job['english_match'] = multi_job['jobtitle'].apply(lambda text:
    process.extractOne(text, esco_need_job_titles_english['preferred_occupation_label'].tolist(), scorer=fuzz.token_set_ratio, score_cutoff=85))

In [23]:
#Matching for own language for all records
multi_job['same_language_match'] = multi_job.apply(text_match_same_language, axis=1)

In [24]:
#handling of null values
multi_job.fillna('mv-9', inplace=True)

In [25]:
# running the cosine similarity for each language
multi_en = cosine_similarity('EN', esco_need_job_titles_english)
multi_hu = cosine_similarity('HU', esco_need_job_titles_hungarian)
multi_de = cosine_similarity('DE', esco_need_job_titles_german)
multi_es = cosine_similarity('ES', esco_need_job_titles_spanish)
multi_sv = cosine_similarity('SV', esco_need_job_titles_sweden)

In [26]:
#Joining back everything to main dataset
multi_job = pd.merge(multi_job, multi_en[['uniq_id', 'cosine_sim_result_EN', 'ngram_sim_EN']], on='uniq_id', how='left')
multi_job = pd.merge(multi_job, multi_hu[['uniq_id', 'cosine_sim_result_HU', 'ngram_sim_HU']], on='uniq_id', how='left')
multi_job = pd.merge(multi_job, multi_de[['uniq_id', 'cosine_sim_result_DE', 'ngram_sim_DE']], on='uniq_id', how='left')
multi_job = pd.merge(multi_job, multi_es[['uniq_id', 'cosine_sim_result_ES', 'ngram_sim_ES']], on='uniq_id', how='left')
multi_job = pd.merge(multi_job, multi_sv[['uniq_id', 'cosine_sim_result_SV', 'ngram_sim_SV']], on='uniq_id', how='left')

In [27]:
#Handling of null values
multi_job.fillna('mv-9',inplace=True)

In [28]:
# select final predictions
multi_job['final_match'] = multi_job.apply(prediction_sorter, axis=1)

In [29]:
# clean final prediction if necessary
multi_job['final_match_cleaned'] = multi_job.apply(prediction_cleaner, axis=1)

In [30]:
#Handling of null values
multi_job.fillna('mv-9', inplace=True)

In [31]:
#writing out complete data (even if no match was found)
multi_job.to_excel('esco_multi_lang_matched_2018_05_04.xlsx')

In [32]:
#Preparing data for visualization. Leaving in data only where there was a match, joining occupation URL's (esco key for positions)
multi_job_vis = multi_job[['uniq_id', 'jobtitle', 'company', 'joblocation_address',  'language', 'final_match_cleaned']][multi_job['final_match_cleaned'] != 'mv-9'].copy()
multi_job_vis['preferred_occupation_label'] = multi_job_vis['final_match_cleaned']
esco_need = esco_need.astype(str)
esco_need = esco_need.applymap(lambda text: text.lower())
occupation_unique = esco_need[['preferred_occupation_label', 'occupationUri']].drop_duplicates()
multi_job_vis = pd.merge(multi_job_vis, occupation_unique, how='left', on='preferred_occupation_label')

In [33]:
#Writing out data for visualization
multi_job_vis.to_excel('esco_matched_for_vis_2018_05_04.xlsx')